# Cross validation of different models

In [20]:
%matplotlib inline
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

from embeddings import *
from tools import *
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer
import os.path

# Load library
from nltk.corpus import stopwords
from gensim import *
import pickle
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import gensim.downloader as api
import re

# Data input and output paths
POS_TRAIN_PATH = '../data/twitter-datasets/train_pos_full.txt' 
NEG_TRAIN_PATH = '../data/twitter-datasets/train_neg_full.txt' 
DATA_TEST_PATH = '../data/twitter-datasets/test_data.txt'
OUTPUT_PATH = 'predictions_out.csv'

TOKENS_PATH = "../saved_gen_files/all_tokens.txt"
FULL_TRAIN_TWEET_VECTORS = "../saved_gen_files/train_tweet_vectors.txt"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard module is not an IPython extension.


## Import data

In [2]:
pos_ids, pos_text_train = load_csv_test_data(POS_TRAIN_PATH)
neg_ids, neg_text_train = load_csv_test_data(NEG_TRAIN_PATH)
full_dataset = np.concatenate((pos_text_train, neg_text_train), axis=None)
full_labels = np.concatenate((np.ones(len(pos_text_train)), -np.ones(len(pos_text_train))), axis=None)

## Tokenize

In [3]:
if os.path.isfile(TOKENS_PATH):
    with open(TOKENS_PATH, 'rb') as f:
        all_tokens = pickle.load(f)
else:    
    all_tokens = [tokenizeTweet(tweet, stop_word=False, 
                           smiley_tag = False, strip_handles=True, 
                           reduce_len=True, preserve_case=False) for tweet in full_dataset]
    
    with open(TOKENS_PATH, 'wb') as f:
        pickle.dump(all_tokens, f)

In [4]:
# Generate bigrams
all_tokens = computeBigrams(all_tokens)

In [5]:
# Force the negative sentiment to be clasified to 0 instead of 1 
labels = full_labels
labels[labels<0] = 0

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(all_tokens, labels, test_size=0.1)

## Word Embedding

Choose one of the embedding algo and the dimensions of the vectors



In [6]:
DIM = 100

### Word2Vec

In [28]:
wv = getWord2VecDict(all_tokens, size=DIM, window=10, min_count=2, workers=10, iters=10, train=True)

You haven't train Word2Vec already with those parameters, it can take some time...


### Glove

In [7]:
wv = api.load("glove-twitter-" + str(DIM)).wv

/home/lucas/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


### Fastext

In [ ]:
# Train a word2vec model to generate embedding
wv = getFasttextDict(all_tokens, size=DIM, window=10, min_count=2, workers=10, iters=10, train=False)

## Model Selection

### Linear Regression

In [ ]:
# Convert tweet in features with previous embedding system
all_tweets_vectors = generateTweetsFeatures(all_tokens, wv)

In [ ]:
# Train and test the model
clf = linear_model.Ridge(alpha=0.1)
#clf = linear_model.LogisticRegression()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predict_labels(predictions)))

### SVM

In [ ]:
from sklearn import svm

clf_svm = svm.SVC(gamma='scale')
clf_svm.fit(X_train, y_train)
predict_svm = clf_svm.predict(X_test)
print(classification_report(y_test, predict_labels(predict_svm)))

### LSTM

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

max_length = max([len(tweet_tokens) for tweet_tokens in X_train])

/home/lucas/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lucas/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lucas/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lucas/anac

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Transform each unique word in unique int identifier
sequences = tokenizer.texts_to_sequences(X_train)


# Pad the tweet to have all the same size
tweet_padded = pad_sequences(sequences, maxlen=max_length)

# Same for test data
test_sequences = tokenizer.texts_to_sequences(X_test)
test_tweet_pad = pad_sequences(test_sequences, maxlen=max_length)

In [ ]:
%tensorboard --logdir logs

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Embedding, Dense, Dropout, Activation, GRU, LSTM, Bidirectional, Flatten, GlobalMaxPool1D

model_name = "BLSTM_100.h5"

model = Sequential()

# Add the embedding layer with our trained embedding matrix
embedding_layer = Embedding(input_dim=wv.syn0.shape[0], 
                            output_dim=wv.syn0.shape[1], 
                            weights=[wv.syn0], 
                            input_length=tweet_padded.shape[1])
model.add(embedding_layer)

# Add dropout to prevent overfitting
model.add(Dropout(0.4))

# Add BiLSTM 
model.add(Bidirectional(LSTM(64, return_sequences=True)))

#self.model.add(GlobalMaxPool1D()) #Or at the same place as Flatten()
model.add(Dense(32))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)

logdir = 'logs/blstm'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)
callbacks = [reduce_lr, tensorboard_callback]

model.summary()

/home/lucas/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':
/home/lucas/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  # Remove the CWD from sys.path while we load stuff.
/home/lucas/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  # This is added back by InteractiveShellApp.init_path()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 85, 100)           119351400 
_________________________________________________________________
dropout (Dropout)            (None, 85, 100)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 85, 128)           84480     
_________________________________________________________________
dense (Dense)                (None, 85, 32)            4128      
_________________________________________________________________
dropout_1 (Dropout)          (None, 85, 32)            0         
_________________________________________________________________
activation (Activation)      (None, 85, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 2720)              0

In [ ]:
#Or we can import the model
model = tf.keras.models.load_model(model_name)
model.summary()

In [11]:
%time model.fit(tweet_padded, y_train, batch_size=512, epochs=10, validation_data=(test_tweet_pad, y_test), shuffle=True, callbacks=callbacks)

Train on 2250000 samples, validate on 250000 samples
Epoch 1/10
2250000/2250000 [==============================] - 7242s 3ms/sample - loss: 0.3790 - accuracy: 0.8200 - val_loss: 0.3169 - val_accuracy: 0.8578
Epoch 2/10
2250000/2250000 [==============================] - 7170s 3ms/sample - loss: 0.3151 - accuracy: 0.8586 - val_loss: 0.2969 - val_accuracy: 0.8689
Epoch 3/10
2250000/2250000 [==============================] - 7171s 3ms/sample - loss: 0.2897 - accuracy: 0.8723 - val_loss: 0.2895 - val_accuracy: 0.8724
Epoch 4/10
2250000/2250000 [==============================] - 7177s 3ms/sample - loss: 0.2715 - accuracy: 0.8819 - val_loss: 0.2881 - val_accuracy: 0.8750
Epoch 5/10
2250000/2250000 [==============================] - 7220s 3ms/sample - loss: 0.2572 - accuracy: 0.8887 - val_loss: 0.2887 - val_accuracy: 0.8746
Epoch 6/10
2250000/2250000 [==============================] - 7337s 3ms/sample - loss: 0.2460 - accuracy: 0.8943 - val_loss: 0.2955 - val_accuracy: 0.8753
Epoch 7/10
225000

In [12]:
#Save the model if needed
model.save(model_name)

In [13]:
# Predict
predictions = model.predict(x=test_tweet_pad)

In [18]:
opt_prob = None
f1_max = 0

for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    f1 = sklearn.metrics.f1_score(y_test, (predictions > thresh).astype(int))
    print('F1 score at threshold {} is {}'.format(thresh, f1), end='\r')
    
    if f1 > f1_max:
        f1_max = f1
        opt_prob = thresh
        
print('Optimal probabilty threshold is {} for maximum F1 score {}'.format(opt_prob, f1_max))

Optimal probabilty threshold is 0.42 for maximum F1 score 0.8796874401501513


In [22]:
print(classification_report(y_test, predict_labels(predictions, opt_prob)))

/home/lucas/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lucas/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         0
         0.0       0.00      0.00      0.00    124879
         1.0       0.84      0.92      0.88    125121

    accuracy                           0.46    250000
   macro avg       0.28      0.31      0.29    250000
weighted avg       0.42      0.46      0.44    250000



## Final Prediction for LSTM

In [23]:
# Retrain the model on the entire dataset
#clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
#clf.fit(all_tweets_vectors, full_labels)

# Load the data to predict
test_ids, test_x = load_csv_test_data(DATA_TEST_PATH, has_ID=True)

# Tokenize it
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False)
test_tokens = [tknzr.tokenize(tweet) for tweet in test_x]

# Generate vector representation
#all_tweets_vectors = np.array([generateTweetVector(model.wv, words) for words in test_tokens])
test_sequences = tokenizer.texts_to_sequences(test_tokens)
test_tweet_pad = pad_sequences(test_sequences, maxlen=max_length)

# Predict
predictions = model.predict(test_tweet_pad)

# Save predictions
create_csv_submission(test_ids, predict_labels(predictions, opt_prob), OUTPUT_PATH)